# 5.1 합성곱 신경망 소개

### MNIST 숫자 이미지 분류에 convnet을 적용

**기본적인 Convnet의 형태**
- **Conv2D**와 **MaxPooling2D**층을 쌓아올린 후 마지막에 **완전연결층(FCN, Fully Connected Network)**을 쌓은 형태
- **Conv2D와 MaxPooling2D층의 출력**
    - **(height, width, channels) 크기의 3D 텐서**
    - 출력의 채널 수는 Conv2D층의 첫번째 매개변수에 의해 조절됨
- 완전연결층에 3D 출력 텐서를 주입해야 하므로 1D 텐서로 펼치는 과정이 있음

In [1]:
# Simple Convnet

from keras import layers
from keras import models

model = models.Sequential()

# Conv2D() + MaxPooling2D()
# input_shape=(image_height, image_width, image_channels)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# 3D -> 1D
model.add(layers.Flatten())

# Dense()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [2]:
# Data Preprocessing

from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255.

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255.

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(60000, 28, 28, 1) (60000, 10)
(10000, 28, 28, 1) (10000, 10)


In [3]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs = 5
hist = model.fit(train_images, train_labels, epochs=epochs, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 5s 91us/step - loss: 0.1622 - accuracy: 0.9506
Epoch 2/5
60000/60000 [==============================] - 4s 72us/step - loss: 0.0447 - accuracy: 0.9862
Epoch 3/5
60000/60000 [==============================] - 5s 82us/step - loss: 0.0307 - accuracy: 0.9907
Epoch 4/5
60000/60000 [==============================] - 5s 89us/step - loss: 0.0235 - accuracy: 0.9929
Epoch 5/5
60000/60000 [==============================] - 5s 84us/step - loss: 0.0178 - accuracy: 0.9944


In [4]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_loss, test_acc)

10000/10000 [==============================] - 1s 51us/step
0.02688317510868801 0.9919000267982483


**MNIST에서 Convnet이 2장의 FCN보다 좋은 테스트 정확도를 보임**

## 5.1.1 합성곱 연산 (Convolution)

### 완전 연결 층과 합성곱 층 사이의 근본적인 차이
- 완전 연결 층
    - 입력 특성 공간에 있는 **전역 패턴**을 학습

- 합성곱 층
    - 입력 공간 특성에 있는 **지역 패턴**을 학습


### Convnet의 2가지 성질
1. 학습된 패턴은 **평행 이동 불변성(translation invariant)**을 가진다.
    - 컨브넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳(예를 들어 왼쪽 위 모서리)에서도 이 패턴을 인식할 수 있음
    - 이 성질은 이미지를 효율적으로 처리하게 만들어 줌
    - 근본적으로 우리가 보는 세상은 **평행 이동으로 인해 다르게 인식되지 않음**
    
<img src="./images/translation_invariance.png" alt="translation_invariance" align="left" style="zoom: 20%" />

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>


2. Convnet은 패턴의 **공간적 계층 구조**를 학습할 수 있다
    - **1번째 합성곱 층**이 edge같은 **작은 지역 패턴**을 학습
    - **2번째 합성곱 층**이 **1번째 합성곱 층의 특성으로 구성된 더 큰 패턴**을 학습
    - 위와 같은 방식으로 convnet은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있음
    - 근본적으로 우리가 보는 세상은 **공간적 계층 구조**를 가짐
    
<img src="./images/spatial_hierarchy.png" alt="spatial_hierarchy" align="left" style="zoom: 75%" />

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>


### 특성 맵(Feature Map)
- 합성곱 연산은 **특성 맵(feature map)**이라 부르는 **3D 텐서(width, height, filters)**에 적용됨
- 합성곱 연산은 **입력 특성 맵에서 작은 패치(patch)들을 추출하고 추출한 모든 패치에 같은 변환을 적용하여 출력 특성 맵(output feature map)**을 만듦
- **특성 맵 텐서의 깊이 축은 필터(filter)를 의미**
    - **필터(filter)란 입력 데이터의 어떠한 특성**을 인코딩하는 것
    - 예를 들어 고수준으로 보면 하나의 필터가 입력에 얼굴이 있는지를 인코딩할 수 있음


- MNIST 예제
    - 1번째 합성곱 층 (28, 28, **1**) : 입력 특성 맵
    - 2번째 합성곱 층 (26, 26, **32**) : 출력 특성 맵
    - 입력에 대해 32개의 필터가 적용되었음
    - **32개의 출력 필터의 각 (26, 26)크기의 배열 값들을 입력에 대한 필터의 응답 맵(response map)**이라고 하며 **입력의 각 위치에서 필터 패턴에 대한 응답**을 나타냄
 

### 합성곱의 2가지 핵심 파라미터

1. 입력으로부터 뽑아낼 **패치의 크기**
2. 특성 맵의 **출력 깊이(합성곱으로 계산 할 필터의 수)**


- 위 2가지 핵심 파라미터는 `Conv2D()`의 1번째, 2번째 매개변수로 전달됨
- `Conv2D(output_depth, (window_height, window_width))`


### 합성곱의 동작

1. **3D 입력 특성 맵**에서 **3D 특성 패치(window_height, window_width, input_depth)를 추출**
    - 3x3 또는 5x5 크기의 **윈도우가 슬라이딩(sliding)**하며 추출


2. **추출된 3D 패치**를 (output_depth,)크기의 **1D 벡터로 변환**
    - **합성곱 커널(convolution kernel)이라 부르는 가중치 행렬과의 텐서 곱셈**을 통해 변환됨)
    - 아래 그림의 경우, 각 3D 패치(3, 3, 2)와 합성곱 커널(3, 3, 2, 3)와의 점곱을 통해 1D 벡터로 패치가 변환됨
    
    
3. **변환된 모든 1D 벡터**는 (height, width, output_depth)크기의 **3D 출력 특성 맵으로 재구성**
    - **출력 특성 맵의 공간상 위치는 입력 특성 맵의 같은 위치**에 대응됨)
  
<img src="./images/how_convolution_works.png" alt="how_convolution_works" align="left" style="zoom: 130%"/>

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>


### 경계 문제와 패딩 이해하기

**경계 문제**
- **입력 특성 맵에서 가능한 패치의 크기에 따른 출력 특성 맵의 크기 감소**를 의미함
- 예를 들어 5x5 입력 특성 맵에서 3x3 패치를 사용하면 출력 특성 맵의 크기가 3x3크기로 줄어듦

<img src="./images/boundary.gif" alt="boundary" align="left" style="zoom: 50%"/>

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>


**패딩(Padding)**
- **입력 특성 맵의 가장자리에 적절한 개수의 행과 열을 추가하는 것을 의미**함
    - 추가되는 행과 열은 0으로 채워지므로 zero padding이라고도 함
    
    
- **입력 특성 맵과 동일한 높이와 너비를 가진 출력 특성 맵**을 얻고 싶을 때 사용

- `Conv2D()`의 `padding` 매개변수로 설정할 수 있음
    - `valid` 설정 시 패딩 사용 X(윈도우를 놓을 수 있는 위치만 사용)
    - `same`  설정 시 입력과 동일한 높이와 너비를 가진 출력이 나오도록 패딩 사용
    - `padding` 매개변수의 기본값은 `valid`

<img src="./images/padding.gif" alt="padding" align="left" style="zoom: 60%"/>

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>


### 합성곱 스트라이드 이해하기

**스트라이드(Stride)**
- **2번의 연속적인 윈도우 사이의 거리(입력 특성 맵에서 패치 추출 시 움직이는 칸 수)**를 스트라이드(stride)라고 함
- 특성 맵을 **다운샘플링(downsampling)** 하기 위한 방법으로도 사용됨
    - stride 2를 사용하면, 특성 맵의 너비와 높이가 2의 배수로 다운샘플링 됨
    
    
- 아래 그림은 stride 2를 적용한 예

<img src="./images/stride.gif" alt="stride" align="left"/>

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>


### 최대 풀링 연산 (Max Pooling)
- 입력 특성 맵에서 패치를 추출하고 각 패치별 **최댓값**으로 구성된 출력 특성 맵을 반환하는 연산


- 합성곱 연산과의 가장 큰 차이점
    - 최대 풀링 연산의 경우 2x2 윈도우와 stride 2를 사용해 **특성 맵을 절반 크기로 downsampling**
    - 합성곱 연산의 경우 주로 3x3 윈도우와 stride 1을 사용

<br/>

**Max Pooling 연산의 역할**
1. **특성의 공간적 계층 구조 학습을 도와줌**
    - Convolution 연산만으로는 초기 입력에 대한 정보가 적음(Convolution 연산의 윈도우 크기만으로 이미지를 보고 분류하는 셈)
    - MaxPooling 연산은 연속적인 Convolution 연산이 점점 **더 커진 윈도우를 통해 바라보도록** 만들어줌
    - **더 커진 윈도우를 통해 전달된 더 많은 정보는 필터의 공간적인 계층 구조를 구성하도록 해줌**


2. **가중치 수를 줄여줌**
    - **Downsampling**을 통해 처리할 특성 맵의 가중치 수가 줄어듦

<br/>
        
**평균 풀링 연산(Average Pooling)**과의 비교
- Average Pooling 연산은 입력 특성 맵에서 패치를 추출하고 각 패치별 **평균값**으로 구성된 출력 특성 맵을 반환하는 연산
- **MaxPooling 연산이 더 잘 동작하는 편임**
    - 특성이 특성 맵의 각 타일에서 어떤 패턴이나 개념의 존재 여부를 인코딩 하는 경향이 있음
    - **평균값을 사용할 경우 정보를 놓치거나 희석시키는 결과가 있음**